#### Imports

In [3]:
import grpc
import tensorflow as tf
from keras_image_helper import create_preprocessor
from tensorflow import keras
from tensorflow_serving.apis import prediction_service_pb2_grpc, predict_pb2

ImportError: cannot import name 'keras' from 'tensorflow' (/Users/roman_didyk/Documents/dev/python/rd-ml-capstone1-2024/.venv/lib/python3.10/site-packages/tensorflow/__init__.py)

In [ ]:
model = keras.models.load_model('./best_model.h5')

tf.saved_model.save(model, 'traffic-sign-model')

In [ ]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [ ]:
preprocessor = create_preprocessor('xception', target_size=(32, 32))

In [ ]:
url = 'https://www.alamy.com/a-traffic-sign-indicating-a-speed-limit-of-50-kmh-seen-near-tubingen-image155445736.html'
X = preprocessor.from_url(url)

In [ ]:
X

In [ ]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [ ]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'traffic-sign-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [ ]:
pb_request

In [ ]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [ ]:
pb_response

In [ ]:
preds = pb_response.outputs['dense_7'].float_val

In [ ]:
classes = [
'Speed limit (20km/h)',
'Speed limit (30km/h)',
'Speed limit (50km/h)',
'Speed limit (60km/h)',
'Speed limit (70km/h)',
'Speed limit (80km/h)',
'End of speed limit (80km/h)',
'Speed limit (100km/h)',
'Speed limit (120km/h)',
'No passing',
'No passing for vehicles over 3.5 metric tons',
'Right-of-way at the next intersection',
'Priority road',
'Yield',
'Stop',
'No vehicles',
'Vehicles over 3.5 metric tons prohibited',
'No entry',
'General caution',
'Dangerous curve to the left',
'Dangerous curve to the right',
'Double curve',
'Bumpy road',
'Slippery road',
'Road narrows on the right',
'Road work',
'Traffic signals',
'Pedestrians',
'Children crossing',
'Bicycles crossing',
'Beware of ice/snow',
'Wild animals crossing',
'End of all speed and passing limits',
'Turn right ahead',
'Turn left ahead',
'Ahead only',
'Go straight or right',
'Go straight or left',
'Keep right',
'Keep left',
'Roundabout mandatory',
'End of no passing',
'End of no passing by vehicles over 3.5 metric'
]

In [ ]:
dict(zip(classes, preds))